In [13]:
#Import modules
import numpy as np
import holidays
import pandas as pd
import pickle

import datetime
import sklearn
import scipy
from scipy import stats
from scipy.stats import ttest_ind
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

from sklearn.model_selection import GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix


import seaborn as sns
import math
from sklearn.model_selection import cross_val_score
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing

import plotly
import plotly.graph_objects as go
import plotly.express as px

#importing some more modules
from sklearn import ensemble
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import cross_val_score

from sklearn.preprocessing import OneHotEncoder
#Import geospatial modules
import geopandas as gpd
from shapely.geometry import Point, Polygon, LineString, shape
import fiona
from gmplot import *
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
import folium 
import plotly

from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
%matplotlib inline
plt.style.use('dark_background')
import warnings
warnings.filterwarnings("ignore")
#train test split
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.utils import resample
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier, ExtraTreesClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
import lightgbm as lgb

In [2]:
#import pickled file

df = pd.read_pickle("df2.pkl")
df.head()

,1st_road_class,1st_road_number,2nd_road_number,accident_severity,carriageway_hazards,date,day_of_week,did_police_officer_attend_scene_of_accident,junction_control,junction_detail,...,vehicle_reference,vehicle_type,was_vehicle_left_hand_drive,x1st_point_of_impact,month,weekend,hour,time_of_day,season,accident_seriousness
accident_index,,,,,,,,,,,,,,,,,,,,,
201001BS70003,B,302.0,0.0,Slight,None,2010-01-11,Monday,Yes,Give way or uncontrolled,T or staggered junction,...,1,Van_Goods_vehicle,No,Front,1,No,7,Morning Rush (6-10),winter,Not Severe
201001BS70004,A,402.0,4204.0,Slight,None,2010-01-11,Monday,Yes,Auto traffic signal,T or staggered junction,...,1,Car,No,Front,1,No,18,Evening (18-22),winter,Not Severe
201001BS70007,Unclassified,0.0,0.0,Slight,None,2010-01-02,Saturday,Yes,Give way or uncontrolled,Mini-roundabout,...,1,Car,No,Nearside,1,Yes,21,Evening (18-22),winter,Not Severe
201001BS70007,Unclassified,0.0,0.0,Slight,None,2010-01-02,Saturday,Yes,Give way or uncontrolled,Mini-roundabout,...,2,Car,No,Front,1,Yes,21,Evening (18-22),winter,Not Severe
201001BS70008,A,3217.0,3220.0,Slight,None,2010-01-04,Monday,Yes,Auto traffic signal,Crossroads,...,1,Car,No,Nearside,1,No,20,Evening (18-22),winter,Not Severe


In [3]:
df1 = df.apply(LabelEncoder().fit_transform)

### Undersampling

In [90]:
#First set up of X and Y
X= df1.drop(['accident_severity','accident_seriousness'],axis=1)
y= df1['accident_seriousness']


In [91]:
# setting up testing and training sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=27)

In [92]:
# concatenate our training data back together
X = pd.concat([X_train, y_train], axis=1)

In [93]:
# separate minority and majority classes
not_severe = X[X.accident_seriousness==0]
severe = X[X.accident_seriousness==1]

In [94]:
# decrease majority
not_severe_decreased = resample(not_severe,
                          replace=True, # sample with replacement
                          n_samples=len(severe), # match number in majority class
                          random_state=27) # reproducible results

In [95]:
# combine majority and severe_increased minority
newdf = pd.concat([severe, not_severe_decreased])

In [96]:
newdf.accident_seriousness.value_counts()

1    51357
0    51357
Name: accident_seriousness, dtype: int64

In [97]:
X_train = newdf.drop('accident_seriousness', axis=1)
y_train = newdf.accident_seriousness

In [ ]:
#scaling
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [98]:
#Decision Tree Classifier

dtc = DecisionTreeClassifier(random_state=42)

dtc.fit(X_train, y_train)
pred_dtc = dtc.predict(X_test)
#Check accuracy

print('\n')
print("Decision Tree Classifier Accuracy Score:",accuracy_score(y_test,pred_dtc )*100)
print("Decision Tree Classifier F1 Score:",f1_score(y_test,pred_dtc, average="macro")*100)
print("Decision Tree Classifier Precision Score:",precision_score(y_test,pred_dtc, average="macro")*100)
print("Decision Tree Classifier Recall Score:",recall_score(y_test, pred_dtc, average="macro")*100) 
print("Decision Tree Classifier Cross Validation Scores:", cross_val_score(dtc, X_train, y_train, cv=3)*100)
print("\n")
print("Decision Tree Classifier Confusion Matrix:\n", confusion_matrix(y_test,pred_dtc))



Decision Tree Classifier Accuracy Score: 57.58104987026318
Decision Tree Classifier F1 Score: 47.921401427620815
Decision Tree Classifier Precision Score: 53.703711236889184
Decision Tree Classifier Recall Score: 58.61286010211422
Decision Tree Classifier Cross Validation Scores: [60.3598341  59.94509025 60.45621824]


Decision Tree Classifier Confusion Matrix:
 [[70597 52713]
 [ 6794 10180]]


In [99]:
#Bagging Classifier
bagc = BaggingClassifier(random_state=42)

bagc.fit(X_train, y_train)
pred_bagc = bagc.predict(X_test)


#Check accuracy

print('\n')
print("Bagging Classifier Accuracy Score:",accuracy_score(y_test,pred_bagc )*100)
print("Bagging Classifier F1 Score:",f1_score(y_test,pred_bagc, average="macro")*100)
print("Bagging Classifier Precision Score:",precision_score(y_test,pred_bagc, average="macro")*100)
print("Bagging Classifier Recall Score:",recall_score(y_test, pred_bagc, average="macro")*100) 
print("Bagging Classifier Cross Validation Scores:", cross_val_score(bagc, X_train, y_train, cv=3)*100)
print("\n")
print("Bagging Classifier Confusion Matrix:\n", confusion_matrix(y_test,pred_bagc))



Bagging Classifier Accuracy Score: 68.22873599270052
Bagging Classifier F1 Score: 54.95006686821211
Bagging Classifier Precision Score: 56.50311232517249
Bagging Classifier Recall Score: 63.64330589919771
Bagging Classifier Cross Validation Scores: [64.13341901 64.13633974 64.78182137]


Bagging Classifier Confusion Matrix:
 [[85938 37372]
 [ 7198  9776]]


In [100]:
#ExtraTreesClassifier

extc = ExtraTreesClassifier(random_state=42)
extc.fit(X_train, y_train)
pred_extc = extc.predict(X_test)

#Check accuracy

print('\n')
print("Extra Trees Classifier Accuracy Score:",accuracy_score(y_test,pred_extc )*100)
print("Extra Trees Classifier F1 Score:",f1_score(y_test,pred_extc, average="macro")*100)
print("Extra Trees Classifier Precision Score:",precision_score(y_test,pred_extc, average="macro")*100)
print("Extra Trees Classifier Recall Score:",recall_score(y_test, pred_extc, average="macro")*100) 
print("Extra Trees Classifier Cross Validation Scores:", cross_val_score(extc, X_train, y_train, cv=3)*100)
print("\n")
print("Extra Trees Classifier Confusion Matrix:\n", confusion_matrix(y_test,pred_extc))




Extra Trees Classifier Accuracy Score: 66.5271877049414
Extra Trees Classifier F1 Score: 53.689224006104965
Extra Trees Classifier Precision Score: 55.855654384248865
Extra Trees Classifier Recall Score: 62.566191503925104
Extra Trees Classifier Cross Validation Scores: [63.63981541 63.45873007 63.55219347]


Extra Trees Classifier Confusion Matrix:
 [[83594 39716]
 [ 7241  9733]]


In [101]:
#AdaBoost Classifier 

adbc = AdaBoostClassifier(random_state=42)
adbc.fit(X_train, y_train)
pred_adbc = adbc.predict(X_test)

#Check accuracy

print('\n')
print("AdaBoost Classifier Accuracy Score:",accuracy_score(y_test,pred_adbc )*100)
print("AdaBoost Classifier F1 Score:",f1_score(y_test,pred_adbc, average="macro")*100)
print("AdaBoost Classifier Precision Score:",precision_score(y_test,pred_adbc, average="macro")*100)
print("AdaBoost Classifier Recall Score:",recall_score(y_test, pred_adbc, average="macro")*100) 
print("AdaBoost Classifier Cross Validation Scores:", cross_val_score(adbc, X_train, y_train, cv=3)*100)
print("\n")
print("AdaBoost Classifier Confusion Matrix:\n", confusion_matrix(y_test,pred_adbc))




AdaBoost Classifier Accuracy Score: 65.86353397393859
AdaBoost Classifier F1 Score: 54.48298144463484
AdaBoost Classifier Precision Score: 57.10604313495624
AdaBoost Classifier Recall Score: 65.72464537323212
AdaBoost Classifier Cross Validation Scores: [65.96471757 65.54413225 65.94135171]


AdaBoost Classifier Confusion Matrix:
 [[81271 42039]
 [ 5849 11125]]


In [102]:
#Random Forest Classifier

rfc = RandomForestClassifier(random_state = 42)
rfc.fit(X_train, y_train)
pred_rfc = rfc.predict(X_test)
#Check accuracy

print('\n')
print("Random Forest Classifier Accuracy Score:",accuracy_score(y_test,pred_rfc )*100)
print("Random Forest Classifier F1 Score:",f1_score(y_test,pred_rfc, average="macro")*100)
print("Random Forest Classifier Precision Score:",precision_score(y_test,pred_rfc, average="macro")*100)
print("Random Forest Classifier Recall Score:",recall_score(y_test, pred_rfc, average="macro")*100) 
print("Random Forest Classifier Cross Validation Scores:", cross_val_score(rfc, X_train, y_train, cv=3)*100)
print("\n")
print("Random Forest Classifier Confusion Matrix:\n", confusion_matrix(y_test,pred_rfc))



Random Forest Classifier Accuracy Score: 68.24370562573066
Random Forest Classifier F1 Score: 54.90640397802291
Random Forest Classifier Precision Score: 56.44945099140549
Random Forest Classifier Recall Score: 63.51211005049373
Random Forest Classifier Cross Validation Scores: [63.73619954 64.10713243 64.77890064]


Random Forest Classifier Confusion Matrix:
 [[86014 37296]
 [ 7253  9721]]


In [104]:
#Gradient Boosting Classifier
gbc = ensemble.GradientBoostingClassifier(random_state = 42)
gbc.fit(X_train, y_train)
pred_gbc = gbc.predict(X_test)
#Check accuracy

print('\n')
print("Gradient Boosting Classifier Accuracy Score:",accuracy_score(y_test,pred_gbc )*100)
print("Gradient Boosting Classifier F1 Score:",f1_score(y_test, pred_gbc, average="macro")*100)
print("Gradient Boosting Classifier Precision Score:",precision_score(y_test,pred_gbc, average="macro")*100)
print("Gradient Boosting Classifier Recall Score:",recall_score(y_test, pred_gbc, average="macro")*100) 
print("Gradient Boosting Classifier Cross Validation Scores:", cross_val_score(gbc, X_train, y_train, cv=3)*100)
print("\n")
print("Gradient Boosting Classifier Confusion Matrix:\n", confusion_matrix(y_test,pred_gbc))




Gradient Boosting Classifier Accuracy Score: 67.33412220923269
Gradient Boosting Classifier F1 Score: 55.52401915609201
Gradient Boosting Classifier Precision Score: 57.56449480319734
Gradient Boosting Classifier Recall Score: 66.45954700265871
Gradient Boosting Classifier Cross Validation Scores: [66.8526199  66.48168701 66.52257725]


Gradient Boosting Classifier Confusion Matrix:
 [[83374 39936]
 [ 5889 11085]]


In [105]:
#Light GBM
lgbm = lgb.LGBMClassifier(random_state = 42)
lgbm.fit(X_train, y_train)
pred_lgbm = lgbm.predict(X_test)

print('\n')
print("LightGBM Classifier Accuracy Score:",accuracy_score(y_test,pred_lgbm )*100)
print("LightGBM Classifier F1 Score:",f1_score(y_test, pred_lgbm, average="macro")*100)
print("LightGBM Classifier Precision Score:",precision_score(y_test,pred_lgbm, average="macro")*100)
print("LightGBM Classifier Recall Score:",recall_score(y_test, pred_lgbm, average="macro")*100) 
print("LightGBM Classifier Cross Validation Scores:", cross_val_score(lgbm, X_train, y_train, cv=3)*100)
print("\n")
print("LightGBM Classifier Confusion Matrix:\n", confusion_matrix(y_test,pred_lgbm))




LightGBM Classifier Accuracy Score: 67.47098742550826
LightGBM Classifier F1 Score: 55.995189092305196
LightGBM Classifier Precision Score: 58.048578817475494
LightGBM Classifier Recall Score: 67.60682356411911
LightGBM Classifier Cross Validation Scores: [67.5886442  67.65582102 67.73468076]


LightGBM Classifier Confusion Matrix:
 [[83145 40165]
 [ 5468 11506]]


In [106]:
#XGBoost
xgb = XGBClassifier(random_state = 42)
xgb.fit(X_train, y_train)

pred_xgb = xgb.predict(X_test)
print('\n')
print("XGBoost Classifier Accuracy Score:",accuracy_score(y_test,pred_xgb )*100)
print("XGBoost Classifier F1 Score:",f1_score(y_test, pred_xgb, average="macro")*100)
print("XGBoost Classifier Precision Score:",precision_score(y_test,pred_xgb, average="macro")*100)
print("XGBoost Classifier Recall Score:",recall_score(y_test, pred_xgb, average="macro")*100) 
print("XGBoost Classifier Cross Validation Scores:", cross_val_score(xgb, X_train, y_train, cv=3)*100)
print("\n")
print("XGBoost Classifier Confusion Matrix:\n", confusion_matrix(y_test,pred_xgb))




XGBoost Classifier Accuracy Score: 67.2977673861595
XGBoost Classifier F1 Score: 55.507881294719766
XGBoost Classifier Precision Score: 57.564245600192265
XGBoost Classifier Recall Score: 66.46934980877842
XGBoost Classifier Cross Validation Scores: [66.77960161 66.45540043 66.7036626 ]


XGBoost Classifier Confusion Matrix:
 [[83311 39999]
 [ 5877 11097]]


In [231]:
#Xa.dtypes

In [250]:
#CatBoost
catb = CatBoostClassifier(eval_metric='Accuracy',use_best_model=True,random_state = 42)
catb.fit(X_train, y_train, eval_set=(X_test,y_test))

Learning rate set to 0.126903
0:	learn: 0.6323189	test: 0.6258447	best: 0.6258447 (0)	total: 119ms	remaining: 1m 59s
1:	learn: 0.6234301	test: 0.7292207	best: 0.7292207 (1)	total: 227ms	remaining: 1m 53s
2:	learn: 0.6338863	test: 0.6410139	best: 0.7292207 (1)	total: 336ms	remaining: 1m 51s
3:	learn: 0.6342271	test: 0.6372430	best: 0.7292207 (1)	total: 444ms	remaining: 1m 50s
4:	learn: 0.6375470	test: 0.6800419	best: 0.7292207 (1)	total: 554ms	remaining: 1m 50s
5:	learn: 0.6399907	test: 0.6710744	best: 0.7292207 (1)	total: 666ms	remaining: 1m 50s
6:	learn: 0.6391728	test: 0.6722221	best: 0.7292207 (1)	total: 769ms	remaining: 1m 49s
7:	learn: 0.6416847	test: 0.6740897	best: 0.7292207 (1)	total: 873ms	remaining: 1m 48s
8:	learn: 0.6436026	test: 0.6738473	best: 0.7292207 (1)	total: 1s	remaining: 1m 50s
9:	learn: 0.6455206	test: 0.6766702	best: 0.7292207 (1)	total: 1.13s	remaining: 1m 51s
10:	learn: 0.6474288	test: 0.6759288	best: 0.7292207 (1)	total: 1.24s	remaining: 1m 51s
11:	learn: 0.64

93:	learn: 0.6785248	test: 0.6735622	best: 0.7292207 (1)	total: 10.5s	remaining: 1m 41s
94:	learn: 0.6787974	test: 0.6739970	best: 0.7292207 (1)	total: 10.6s	remaining: 1m 41s
95:	learn: 0.6793426	test: 0.6746386	best: 0.7292207 (1)	total: 10.8s	remaining: 1m 41s
96:	learn: 0.6799463	test: 0.6748810	best: 0.7292207 (1)	total: 10.9s	remaining: 1m 41s
97:	learn: 0.6800534	test: 0.6750164	best: 0.7292207 (1)	total: 11s	remaining: 1m 41s
98:	learn: 0.6802481	test: 0.6750806	best: 0.7292207 (1)	total: 11.1s	remaining: 1m 40s
99:	learn: 0.6803844	test: 0.6753087	best: 0.7292207 (1)	total: 11.2s	remaining: 1m 40s
100:	learn: 0.6803746	test: 0.6752873	best: 0.7292207 (1)	total: 11.3s	remaining: 1m 40s
101:	learn: 0.6806667	test: 0.6751946	best: 0.7292207 (1)	total: 11.4s	remaining: 1m 40s
102:	learn: 0.6808614	test: 0.6752730	best: 0.7292207 (1)	total: 11.5s	remaining: 1m 40s
103:	learn: 0.6806764	test: 0.6750520	best: 0.7292207 (1)	total: 11.6s	remaining: 1m 39s
104:	learn: 0.6808906	test: 0.

186:	learn: 0.6925443	test: 0.6796213	best: 0.7292207 (1)	total: 21.2s	remaining: 1m 32s
187:	learn: 0.6926709	test: 0.6795786	best: 0.7292207 (1)	total: 21.3s	remaining: 1m 31s
188:	learn: 0.6930603	test: 0.6795429	best: 0.7292207 (1)	total: 21.4s	remaining: 1m 31s
189:	learn: 0.6933524	test: 0.6795358	best: 0.7292207 (1)	total: 21.5s	remaining: 1m 31s
190:	learn: 0.6932745	test: 0.6795928	best: 0.7292207 (1)	total: 21.6s	remaining: 1m 31s
191:	learn: 0.6933914	test: 0.6795429	best: 0.7292207 (1)	total: 21.7s	remaining: 1m 31s
192:	learn: 0.6935861	test: 0.6795287	best: 0.7292207 (1)	total: 21.9s	remaining: 1m 31s
193:	learn: 0.6939268	test: 0.6798993	best: 0.7292207 (1)	total: 22s	remaining: 1m 31s
194:	learn: 0.6940534	test: 0.6800134	best: 0.7292207 (1)	total: 22.1s	remaining: 1m 31s
195:	learn: 0.6942870	test: 0.6801560	best: 0.7292207 (1)	total: 22.2s	remaining: 1m 31s
196:	learn: 0.6943065	test: 0.6802059	best: 0.7292207 (1)	total: 22.3s	remaining: 1m 31s
197:	learn: 0.6944818	t

280:	learn: 0.7035944	test: 0.6806906	best: 0.7292207 (1)	total: 32.2s	remaining: 1m 22s
281:	learn: 0.7036821	test: 0.6807476	best: 0.7292207 (1)	total: 32.3s	remaining: 1m 22s
282:	learn: 0.7037989	test: 0.6806193	best: 0.7292207 (1)	total: 32.4s	remaining: 1m 22s
283:	learn: 0.7037892	test: 0.6807833	best: 0.7292207 (1)	total: 32.6s	remaining: 1m 22s
284:	learn: 0.7038086	test: 0.6808047	best: 0.7292207 (1)	total: 32.7s	remaining: 1m 22s
285:	learn: 0.7040326	test: 0.6809543	best: 0.7292207 (1)	total: 32.8s	remaining: 1m 21s
286:	learn: 0.7041299	test: 0.6808047	best: 0.7292207 (1)	total: 32.9s	remaining: 1m 21s
287:	learn: 0.7040715	test: 0.6808403	best: 0.7292207 (1)	total: 33s	remaining: 1m 21s
288:	learn: 0.7041981	test: 0.6808902	best: 0.7292207 (1)	total: 33.2s	remaining: 1m 21s
289:	learn: 0.7042467	test: 0.6808973	best: 0.7292207 (1)	total: 33.3s	remaining: 1m 21s
290:	learn: 0.7043052	test: 0.6811254	best: 0.7292207 (1)	total: 33.4s	remaining: 1m 21s
291:	learn: 0.7043149	t

374:	learn: 0.7129213	test: 0.6815246	best: 0.7292207 (1)	total: 43.7s	remaining: 1m 12s
375:	learn: 0.7130868	test: 0.6814676	best: 0.7292207 (1)	total: 43.9s	remaining: 1m 12s
376:	learn: 0.7130284	test: 0.6814106	best: 0.7292207 (1)	total: 44s	remaining: 1m 12s
377:	learn: 0.7130674	test: 0.6815175	best: 0.7292207 (1)	total: 44.1s	remaining: 1m 12s
378:	learn: 0.7131550	test: 0.6814605	best: 0.7292207 (1)	total: 44.2s	remaining: 1m 12s
379:	learn: 0.7133302	test: 0.6815816	best: 0.7292207 (1)	total: 44.3s	remaining: 1m 12s
380:	learn: 0.7133400	test: 0.6816030	best: 0.7292207 (1)	total: 44.5s	remaining: 1m 12s
381:	learn: 0.7133107	test: 0.6815317	best: 0.7292207 (1)	total: 44.6s	remaining: 1m 12s
382:	learn: 0.7130868	test: 0.6816814	best: 0.7292207 (1)	total: 44.7s	remaining: 1m 12s
383:	learn: 0.7131550	test: 0.6817100	best: 0.7292207 (1)	total: 44.8s	remaining: 1m 11s
384:	learn: 0.7134373	test: 0.6817100	best: 0.7292207 (1)	total: 44.9s	remaining: 1m 11s
385:	learn: 0.7135055	t

...

563:	learn: 0.7291801	test: 0.6815175	best: 0.7292207 (1)	total: 1m 6s	remaining: 51.1s
564:	learn: 0.7291703	test: 0.6814676	best: 0.7292207 (1)	total: 1m 6s	remaining: 51s
565:	learn: 0.7291898	test: 0.6813393	best: 0.7292207 (1)	total: 1m 6s	remaining: 50.9s
566:	learn: 0.7292774	test: 0.6815032	best: 0.7292207 (1)	total: 1m 6s	remaining: 50.7s
567:	learn: 0.7290924	test: 0.6814034	best: 0.7292207 (1)	total: 1m 6s	remaining: 50.6s
568:	learn: 0.7292969	test: 0.6813749	best: 0.7292207 (1)	total: 1m 6s	remaining: 50.5s
569:	learn: 0.7294137	test: 0.6812181	best: 0.7292207 (1)	total: 1m 6s	remaining: 50.4s
570:	learn: 0.7295500	test: 0.6810755	best: 0.7292207 (1)	total: 1m 6s	remaining: 50.3s
571:	learn: 0.7295987	test: 0.6812395	best: 0.7292207 (1)	total: 1m 7s	remaining: 50.1s
572:	learn: 0.7294819	test: 0.6810541	best: 0.7292207 (1)	total: 1m 7s	remaining: 50s
573:	learn: 0.7295500	test: 0.6810755	best: 0.7292207 (1)	total: 1m 7s	remaining: 49.9s
574:	learn: 0.7296863	test: 0.681161

657:	learn: 0.7362774	test: 0.6810399	best: 0.7292207 (1)	total: 1m 16s	remaining: 39.9s
658:	learn: 0.7364137	test: 0.6810114	best: 0.7292207 (1)	total: 1m 16s	remaining: 39.8s
659:	learn: 0.7366474	test: 0.6809401	best: 0.7292207 (1)	total: 1m 17s	remaining: 39.7s
660:	learn: 0.7366863	test: 0.6809045	best: 0.7292207 (1)	total: 1m 17s	remaining: 39.6s
661:	learn: 0.7366669	test: 0.6808617	best: 0.7292207 (1)	total: 1m 17s	remaining: 39.4s
662:	learn: 0.7366961	test: 0.6810399	best: 0.7292207 (1)	total: 1m 17s	remaining: 39.3s
663:	learn: 0.7368226	test: 0.6809900	best: 0.7292207 (1)	total: 1m 17s	remaining: 39.2s
664:	learn: 0.7366182	test: 0.6808189	best: 0.7292207 (1)	total: 1m 17s	remaining: 39.1s
665:	learn: 0.7366961	test: 0.6806763	best: 0.7292207 (1)	total: 1m 17s	remaining: 39s
666:	learn: 0.7368129	test: 0.6808189	best: 0.7292207 (1)	total: 1m 17s	remaining: 38.9s
667:	learn: 0.7369005	test: 0.6807904	best: 0.7292207 (1)	total: 1m 17s	remaining: 38.7s
668:	learn: 0.7370758	t

751:	learn: 0.7426154	test: 0.6805552	best: 0.7292207 (1)	total: 1m 27s	remaining: 29s
752:	learn: 0.7427420	test: 0.6805694	best: 0.7292207 (1)	total: 1m 27s	remaining: 28.9s
753:	learn: 0.7428685	test: 0.6804981	best: 0.7292207 (1)	total: 1m 28s	remaining: 28.8s
754:	learn: 0.7429270	test: 0.6804126	best: 0.7292207 (1)	total: 1m 28s	remaining: 28.6s
755:	learn: 0.7430925	test: 0.6804126	best: 0.7292207 (1)	total: 1m 28s	remaining: 28.5s
756:	learn: 0.7430730	test: 0.6802843	best: 0.7292207 (1)	total: 1m 28s	remaining: 28.4s
757:	learn: 0.7431898	test: 0.6803841	best: 0.7292207 (1)	total: 1m 28s	remaining: 28.3s
758:	learn: 0.7431606	test: 0.6804197	best: 0.7292207 (1)	total: 1m 28s	remaining: 28.2s
759:	learn: 0.7430925	test: 0.6804910	best: 0.7292207 (1)	total: 1m 28s	remaining: 28.1s
760:	learn: 0.7430438	test: 0.6804554	best: 0.7292207 (1)	total: 1m 28s	remaining: 27.9s
761:	learn: 0.7431898	test: 0.6804910	best: 0.7292207 (1)	total: 1m 29s	remaining: 27.8s
762:	learn: 0.7434624	t

845:	learn: 0.7490216	test: 0.6805124	best: 0.7292207 (1)	total: 1m 39s	remaining: 18s
846:	learn: 0.7490702	test: 0.6802914	best: 0.7292207 (1)	total: 1m 39s	remaining: 17.9s
847:	learn: 0.7492260	test: 0.6802843	best: 0.7292207 (1)	total: 1m 39s	remaining: 17.8s
848:	learn: 0.7493136	test: 0.6802273	best: 0.7292207 (1)	total: 1m 39s	remaining: 17.7s
849:	learn: 0.7493818	test: 0.6800562	best: 0.7292207 (1)	total: 1m 39s	remaining: 17.6s
850:	learn: 0.7494597	test: 0.6801203	best: 0.7292207 (1)	total: 1m 39s	remaining: 17.5s
851:	learn: 0.7495570	test: 0.6799136	best: 0.7292207 (1)	total: 1m 39s	remaining: 17.4s
852:	learn: 0.7494986	test: 0.6797782	best: 0.7292207 (1)	total: 1m 40s	remaining: 17.2s
853:	learn: 0.7495570	test: 0.6797853	best: 0.7292207 (1)	total: 1m 40s	remaining: 17.1s
854:	learn: 0.7498296	test: 0.6797354	best: 0.7292207 (1)	total: 1m 40s	remaining: 17s
855:	learn: 0.7498199	test: 0.6798067	best: 0.7292207 (1)	total: 1m 40s	remaining: 16.9s
856:	learn: 0.7498394	tes

939:	learn: 0.7555445	test: 0.6800847	best: 0.7292207 (1)	total: 1m 50s	remaining: 7.05s
940:	learn: 0.7556321	test: 0.6800633	best: 0.7292207 (1)	total: 1m 50s	remaining: 6.93s
941:	learn: 0.7556419	test: 0.6800063	best: 0.7292207 (1)	total: 1m 50s	remaining: 6.81s
942:	learn: 0.7558463	test: 0.6798993	best: 0.7292207 (1)	total: 1m 50s	remaining: 6.7s
943:	learn: 0.7559145	test: 0.6798922	best: 0.7292207 (1)	total: 1m 50s	remaining: 6.58s
944:	learn: 0.7560995	test: 0.6798423	best: 0.7292207 (1)	total: 1m 51s	remaining: 6.46s
945:	learn: 0.7560800	test: 0.6798708	best: 0.7292207 (1)	total: 1m 51s	remaining: 6.34s
946:	learn: 0.7561189	test: 0.6799849	best: 0.7292207 (1)	total: 1m 51s	remaining: 6.23s
947:	learn: 0.7561773	test: 0.6798851	best: 0.7292207 (1)	total: 1m 51s	remaining: 6.11s
948:	learn: 0.7564402	test: 0.6797568	best: 0.7292207 (1)	total: 1m 51s	remaining: 5.99s
949:	learn: 0.7564986	test: 0.6798708	best: 0.7292207 (1)	total: 1m 51s	remaining: 5.88s
950:	learn: 0.7564792	

In [252]:
pred_catb = catb.predict(X_test)

print("CatBoost Best Score",catb.get_best_score())
print('\n')
print("CatBoost Classifier Accuracy Score:",accuracy_score(y_test,pred_catb )*100)
print("CatBoost Classifier F1 Score:",f1_score(y_test, pred_catb, average="macro")*100)
print("CatBoost Classifier Precision Score:",precision_score(y_test,pred_catb, average="macro")*100)
print("CatBoost Classifier Recall Score:",recall_score(y_test, pred_catb, average="macro")*100) 
#print("CatBoost Classifier Cross Validation Scores:", cross_val_score(catb, X_train, y_train, cv=3)*100)
print("\n")
print("CatBoost Classifier Confusion Matrix:\n", confusion_matrix(y_test,pred_catb))

CatBoost Best Score {'learn': {'Accuracy': 0.7598672040812353, 'Logloss': 0.5096101938830258}, 'validation': {'Accuracy': 0.729220723674831, 'Logloss': 0.5987517133226292}}


CatBoost Classifier Accuracy Score: 72.92207236748311
CatBoost Classifier F1 Score: 56.5061757999898
CatBoost Classifier Precision Score: 56.528875275509016
CatBoost Classifier Recall Score: 61.94639720370893


CatBoost Classifier Confusion Matrix:
 [[94241 29069]
 [ 8917  8057]]


### Tuning

In [264]:
#bagging param

bag_params = {
    'n_estimators': [10, 100, 200, 300],
    'max_samples': [1, 10, 100, 1000],
    'oob_score': ['true','false'],
    'bootstrap_features': ['true', 'false'],
    'bootstrap': ['true', 'false'],
    'random_state':[42]}

grid_bag = GridSearchCV(bagc, param_grid = bag_params, cv = 3, verbose = 5)
grid_bag.fit(X_train,y_train)


print(bagbest_estimator = grid_bag.best_estimator_)
print("Bagging Best Params:\n",grid_bag.best_params_)  

Fitting 3 folds for each of 128 candidates, totalling 384 fits
[CV] bootstrap=true, bootstrap_features=true, max_samples=1, n_estimators=10, oob_score=true, random_state=42 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  bootstrap=true, bootstrap_features=true, max_samples=1, n_estimators=10, oob_score=true, random_state=42, score=0.500, total=   2.1s
[CV] bootstrap=true, bootstrap_features=true, max_samples=1, n_estimators=10, oob_score=true, random_state=42 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.0s remaining:    0.0s


[CV]  bootstrap=true, bootstrap_features=true, max_samples=1, n_estimators=10, oob_score=true, random_state=42, score=0.500, total=   2.2s
[CV] bootstrap=true, bootstrap_features=true, max_samples=1, n_estimators=10, oob_score=true, random_state=42 


...

[CV]  bootstrap=false, bootstrap_features=false, max_samples=1, n_estimators=100, oob_score=true, random_state=42, score=0.500, total=  21.8s
[CV] bootstrap=false, bootstrap_features=false, max_samples=1, n_estimators=100, oob_score=false, random_state=42 
[CV]  bootstrap=false, bootstrap_features=false, max_samples=1, n_estimators=100, oob_score=false, random_state=42, score=0.500, total=  23.2s
[CV] bootstrap=false, bootstrap_features=false, max_samples=1, n_estimators=100, oob_score=false, random_state=42 
[CV]  bootstrap=false, bootstrap_features=false, max_samples=1, n_estimators=100, oob_score=false, random_state=42, score=0.500, total=  18.1s
[CV] bootstrap=false, bootstrap_features=false, max_samples=1, n_estimators=100, oob_score=false, random_state=42 
[CV]  bootstrap=false, bootstrap_features=false, max_samples=1, n_estimators=100, oob_score=false, random_state=42, score=0.500, total=  16.6s
[CV] bootstrap=false, bootstrap_features=false, max_samples=1, n_estimators=200, oob

[CV]  bootstrap=false, bootstrap_features=false, max_samples=10, n_estimators=200, oob_score=false, random_state=42, score=0.552, total=  38.2s
[CV] bootstrap=false, bootstrap_features=false, max_samples=10, n_estimators=200, oob_score=false, random_state=42 
[CV]  bootstrap=false, bootstrap_features=false, max_samples=10, n_estimators=200, oob_score=false, random_state=42, score=0.574, total=  36.3s
[CV] bootstrap=false, bootstrap_features=false, max_samples=10, n_estimators=300, oob_score=true, random_state=42 
[CV]  bootstrap=false, bootstrap_features=false, max_samples=10, n_estimators=300, oob_score=true, random_state=42, score=0.584, total=  56.3s
[CV] bootstrap=false, bootstrap_features=false, max_samples=10, n_estimators=300, oob_score=true, random_state=42 
[CV]  bootstrap=false, bootstrap_features=false, max_samples=10, n_estimators=300, oob_score=true, random_state=42, score=0.566, total= 1.0min
[CV] bootstrap=false, bootstrap_features=false, max_samples=10, n_estimators=300

[CV]  bootstrap=false, bootstrap_features=false, max_samples=1000, n_estimators=10, oob_score=true, random_state=42, score=0.592, total=   3.5s
[CV] bootstrap=false, bootstrap_features=false, max_samples=1000, n_estimators=10, oob_score=true, random_state=42 
[CV]  bootstrap=false, bootstrap_features=false, max_samples=1000, n_estimators=10, oob_score=true, random_state=42, score=0.584, total=   3.1s
[CV] bootstrap=false, bootstrap_features=false, max_samples=1000, n_estimators=10, oob_score=true, random_state=42 
[CV]  bootstrap=false, bootstrap_features=false, max_samples=1000, n_estimators=10, oob_score=true, random_state=42, score=0.582, total=   3.2s
[CV] bootstrap=false, bootstrap_features=false, max_samples=1000, n_estimators=10, oob_score=false, random_state=42 
[CV]  bootstrap=false, bootstrap_features=false, max_samples=1000, n_estimators=10, oob_score=false, random_state=42, score=0.592, total=   3.2s
[CV] bootstrap=false, bootstrap_features=false, max_samples=1000, n_estima

[Parallel(n_jobs=1)]: Done 384 out of 384 | elapsed: 210.9min finished


Bagging Best Params:
 {'bootstrap': 'true', 'bootstrap_features': 'true', 'max_samples': 1000, 'n_estimators': 300, 'oob_score': 'true', 'random_state': 42}


In [267]:
#extra trees param
extc_params = {
    'criterion' : ['gini', 'entropy'],
    'max_depth': [10, 50, 100],
    'min_samples_split': [2, 4, 6, 8],
    'oob_score': ['true','false'],
    'bootstrap': ['true', 'false'],
    'random_state':[42]}

grid_extc = GridSearchCV(extc, param_grid = extc_params, cv = 3, verbose = 5)
grid_extc.fit(X_train,y_train)

print(extcbest_estimator = grid_extc.best_estimator_)
print("Extra Trees Best Params:\n",grid_extc.best_params_)  

Fitting 3 folds for each of 96 candidates, totalling 288 fits
[CV] bootstrap=true, criterion=gini, max_depth=10, min_samples_split=2, oob_score=true, random_state=42 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  bootstrap=true, criterion=gini, max_depth=10, min_samples_split=2, oob_score=true, random_state=42, score=0.647, total=   1.2s
[CV] bootstrap=true, criterion=gini, max_depth=10, min_samples_split=2, oob_score=true, random_state=42 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.1s remaining:    0.0s


[CV]  bootstrap=true, criterion=gini, max_depth=10, min_samples_split=2, oob_score=true, random_state=42, score=0.639, total=   1.2s
[CV] bootstrap=true, criterion=gini, max_depth=10, min_samples_split=2, oob_score=true, random_state=42 


...

[CV]  bootstrap=false, criterion=gini, max_depth=100, min_samples_split=8, oob_score=true, random_state=42, score=0.643, total=   3.5s
[CV] bootstrap=false, criterion=gini, max_depth=100, min_samples_split=8, oob_score=true, random_state=42 
[CV]  bootstrap=false, criterion=gini, max_depth=100, min_samples_split=8, oob_score=true, random_state=42, score=0.643, total=   4.9s
[CV] bootstrap=false, criterion=gini, max_depth=100, min_samples_split=8, oob_score=true, random_state=42 
[CV]  bootstrap=false, criterion=gini, max_depth=100, min_samples_split=8, oob_score=true, random_state=42, score=0.644, total=   3.5s
[CV] bootstrap=false, criterion=gini, max_depth=100, min_samples_split=8, oob_score=false, random_state=42 
[CV]  bootstrap=false, criterion=gini, max_depth=100, min_samples_split=8, oob_score=false, random_state=42, score=0.643, total=   3.6s
[CV] bootstrap=false, criterion=gini, max_depth=100, min_samples_split=8, oob_score=false, random_state=42 
[CV]  bootstrap=false, criter

[CV]  bootstrap=false, criterion=entropy, max_depth=50, min_samples_split=2, oob_score=false, random_state=42, score=0.631, total=   4.3s
[CV] bootstrap=false, criterion=entropy, max_depth=50, min_samples_split=2, oob_score=false, random_state=42 
[CV]  bootstrap=false, criterion=entropy, max_depth=50, min_samples_split=2, oob_score=false, random_state=42, score=0.632, total=   4.6s
[CV] bootstrap=false, criterion=entropy, max_depth=50, min_samples_split=4, oob_score=true, random_state=42 
[CV]  bootstrap=false, criterion=entropy, max_depth=50, min_samples_split=4, oob_score=true, random_state=42, score=0.636, total=   5.1s
[CV] bootstrap=false, criterion=entropy, max_depth=50, min_samples_split=4, oob_score=true, random_state=42 
[CV]  bootstrap=false, criterion=entropy, max_depth=50, min_samples_split=4, oob_score=true, random_state=42, score=0.635, total=   4.2s
[CV] bootstrap=false, criterion=entropy, max_depth=50, min_samples_split=4, oob_score=true, random_state=42 
[CV]  bootstr

[CV]  bootstrap=false, criterion=entropy, max_depth=100, min_samples_split=6, oob_score=true, random_state=42, score=0.639, total=   5.6s
[CV] bootstrap=false, criterion=entropy, max_depth=100, min_samples_split=6, oob_score=false, random_state=42 
[CV]  bootstrap=false, criterion=entropy, max_depth=100, min_samples_split=6, oob_score=false, random_state=42, score=0.639, total=   4.2s
[CV] bootstrap=false, criterion=entropy, max_depth=100, min_samples_split=6, oob_score=false, random_state=42 
[CV]  bootstrap=false, criterion=entropy, max_depth=100, min_samples_split=6, oob_score=false, random_state=42, score=0.638, total=   4.1s
[CV] bootstrap=false, criterion=entropy, max_depth=100, min_samples_split=6, oob_score=false, random_state=42 
[CV]  bootstrap=false, criterion=entropy, max_depth=100, min_samples_split=6, oob_score=false, random_state=42, score=0.639, total=   5.8s
[CV] bootstrap=false, criterion=entropy, max_depth=100, min_samples_split=8, oob_score=true, random_state=42 
[C

[Parallel(n_jobs=1)]: Done 288 out of 288 | elapsed: 16.8min finished


Extra Trees Best Params:
 {'bootstrap': 'true', 'criterion': 'gini', 'max_depth': 10, 'min_samples_split': 2, 'oob_score': 'true', 'random_state': 42}


In [268]:
#adaboost param

adbc_params = {'algorithm': ['SAMME','SAMME.R'], 'random_state':[42]}

grid_adbc = GridSearchCV(adbc, param_grid = adbc_params, cv = 3, verbose = 5)
grid_adbc.fit(X_train,y_train)

print(adbcbest_estimator = grid_adbc.best_estimator_)
print("AdaBoost Best Params:\n",grid_adbc.best_params_)  

Fitting 3 folds for each of 2 candidates, totalling 6 fits
[CV] algorithm=SAMME, random_state=42 ................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] .... algorithm=SAMME, random_state=42, score=0.653, total=  14.2s
[CV] algorithm=SAMME, random_state=42 ................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   14.1s remaining:    0.0s


[CV] .... algorithm=SAMME, random_state=42, score=0.647, total=  15.4s
[CV] algorithm=SAMME, random_state=42 ................................


...

[CV] .. algorithm=SAMME.R, random_state=42, score=0.660, total=  18.5s
[CV] algorithm=SAMME.R, random_state=42 ..............................


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:  1.1min remaining:    0.0s


[CV] .. algorithm=SAMME.R, random_state=42, score=0.655, total=  16.2s
[CV] algorithm=SAMME.R, random_state=42 ..............................
[CV] .. algorithm=SAMME.R, random_state=42, score=0.659, total=  22.8s


[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:  1.7min finished


AdaBoost Best Params:
 {'algorithm': 'SAMME.R', 'random_state': 42}


In [254]:
#RANDOM FOREST PARAM
param_grid = {
    'n_estimators': [100, 200, 300],
    'criterion': ['entropy', 'gini'],
    'max_depth': [10, 50, 100],
    'random_state':[42]}

grid_rfc = GridSearchCV(rfc, param_grid = param_grid, cv = 3, verbose = 5)
grid_rfc.fit(X_train,y_train)

print(rfcbest_estimator = grid_rfc.best_estimator_)
print("Random Forest:\n",grid_rfc.best_params_)

Fitting 3 folds for each of 216 candidates, totalling 648 fits
[CV] criterion=entropy, max_depth=10, max_features=log2, min_samples_split=2, n_estimators=100, random_state=42 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  criterion=entropy, max_depth=10, max_features=log2, min_samples_split=2, n_estimators=100, random_state=42, score=0.663, total=   8.9s
[CV] criterion=entropy, max_depth=10, max_features=log2, min_samples_split=2, n_estimators=100, random_state=42 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    8.8s remaining:    0.0s


[CV]  criterion=entropy, max_depth=10, max_features=log2, min_samples_split=2, n_estimators=100, random_state=42, score=0.659, total=   9.6s
[CV] criterion=entropy, max_depth=10, max_features=log2, min_samples_split=2, n_estimators=100, random_state=42 


...

[CV]  criterion=gini, max_depth=50, max_features=auto, min_samples_split=8, n_estimators=300, random_state=42, score=0.688, total= 1.3min
[CV] criterion=gini, max_depth=50, max_features=auto, min_samples_split=8, n_estimators=300, random_state=42 
[CV]  criterion=gini, max_depth=50, max_features=auto, min_samples_split=8, n_estimators=300, random_state=42, score=0.691, total= 1.5min
[CV] criterion=gini, max_depth=100, max_features=log2, min_samples_split=2, n_estimators=100, random_state=42 
[CV]  criterion=gini, max_depth=100, max_features=log2, min_samples_split=2, n_estimators=100, random_state=42, score=0.684, total=  28.1s
[CV] criterion=gini, max_depth=100, max_features=log2, min_samples_split=2, n_estimators=100, random_state=42 
[CV]  criterion=gini, max_depth=100, max_features=log2, min_samples_split=2, n_estimators=100, random_state=42, score=0.682, total=  28.9s
[CV] criterion=gini, max_depth=100, max_features=log2, min_samples_split=2, n_estimators=100, random_state=42 
[CV

[CV]  criterion=gini, max_depth=100, max_features=log2, min_samples_split=8, n_estimators=200, random_state=42, score=0.687, total=  41.7s
[CV] criterion=gini, max_depth=100, max_features=log2, min_samples_split=8, n_estimators=200, random_state=42 
[CV]  criterion=gini, max_depth=100, max_features=log2, min_samples_split=8, n_estimators=200, random_state=42, score=0.688, total=  41.9s
[CV] criterion=gini, max_depth=100, max_features=log2, min_samples_split=8, n_estimators=300, random_state=42 
[CV]  criterion=gini, max_depth=100, max_features=log2, min_samples_split=8, n_estimators=300, random_state=42, score=0.689, total= 1.0min
[CV] criterion=gini, max_depth=100, max_features=log2, min_samples_split=8, n_estimators=300, random_state=42 
[CV]  criterion=gini, max_depth=100, max_features=log2, min_samples_split=8, n_estimators=300, random_state=42, score=0.690, total= 1.0min
[CV] criterion=gini, max_depth=100, max_features=log2, min_samples_split=8, n_estimators=300, random_state=42 


[CV]  criterion=gini, max_depth=100, max_features=sqrt, min_samples_split=8, n_estimators=100, random_state=42, score=0.683, total=  21.2s
[CV] criterion=gini, max_depth=100, max_features=sqrt, min_samples_split=8, n_estimators=100, random_state=42 
[CV]  criterion=gini, max_depth=100, max_features=sqrt, min_samples_split=8, n_estimators=100, random_state=42, score=0.687, total=  21.2s
[CV] criterion=gini, max_depth=100, max_features=sqrt, min_samples_split=8, n_estimators=200, random_state=42 
[CV]  criterion=gini, max_depth=100, max_features=sqrt, min_samples_split=8, n_estimators=200, random_state=42, score=0.688, total=  43.2s
[CV] criterion=gini, max_depth=100, max_features=sqrt, min_samples_split=8, n_estimators=200, random_state=42 
[CV]  criterion=gini, max_depth=100, max_features=sqrt, min_samples_split=8, n_estimators=200, random_state=42, score=0.686, total=  42.5s
[CV] criterion=gini, max_depth=100, max_features=sqrt, min_samples_split=8, n_estimators=200, random_state=42 


[CV]  criterion=gini, max_depth=100, max_features=auto, min_samples_split=6, n_estimators=300, random_state=42, score=0.690, total= 1.1min
[CV] criterion=gini, max_depth=100, max_features=auto, min_samples_split=6, n_estimators=300, random_state=42 
[CV]  criterion=gini, max_depth=100, max_features=auto, min_samples_split=6, n_estimators=300, random_state=42, score=0.694, total= 1.1min
[CV] criterion=gini, max_depth=100, max_features=auto, min_samples_split=8, n_estimators=100, random_state=42 
[CV]  criterion=gini, max_depth=100, max_features=auto, min_samples_split=8, n_estimators=100, random_state=42, score=0.685, total=  22.5s
[CV] criterion=gini, max_depth=100, max_features=auto, min_samples_split=8, n_estimators=100, random_state=42 
[CV]  criterion=gini, max_depth=100, max_features=auto, min_samples_split=8, n_estimators=100, random_state=42, score=0.683, total=  27.1s
[CV] criterion=gini, max_depth=100, max_features=auto, min_samples_split=8, n_estimators=100, random_state=42 


[Parallel(n_jobs=1)]: Done 648 out of 648 | elapsed: 483.0min finished


In [256]:
#Gradient Boosting Classifier Tuning
gbcparam= {'learning_rate':[0.15,0.1,0.05,0.01,0.005,0.001], 
           'n_estimators':[100,250,500,],
           'max_depth': [4, 6, 8],
           'min_samples_leaf': [1, 10, 20, 50, 100],
           'random_state':[42]}

gbctuning = GridSearchCV(estimator = ensemble.GradientBoostingClassifier(min_samples_split=2,
                                                                         subsample=1,
                                                                         max_features='sqrt', 
                                                                         random_state=10), 
#param_grid = gbcparam, scoring='accuracy',n_jobs=4,iid=False, cv=3)


gbctuning.fit(X_train,y_train)
print(gbcbest_estimator = gbctuning.best_estimator_)                         
print("Gradient Boost:\n",gbctuning.best_params_)

Gradient Boost:
 {'learning_rate': 0.05, 'max_depth': 8, 'min_samples_leaf': 1, 'n_estimators': 500, 'random_state': 42}


In [275]:
#Decision Tree Classifier
dtc_param = {'class_weight':['balanced'], 
             'criterion': ['gini','entropy'],
             'max_depth': [4, 6, 8],
             'random_state':[42]}
    

dtctuning = GridSearchCV(dtc, param_grid = dtc_param, cv = 3, verbose = 5)

dtctuning.fit(X_train,y_train)


print(dtcbest_estimator = dtctuning.best_estimator_)
print("Decision Tree Best Params:\n",dtctuning.best_params_)

Fitting 3 folds for each of 6 candidates, totalling 18 fits
[CV] class_weight=balanced, criterion=gini, max_depth=4, random_state=42 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  class_weight=balanced, criterion=gini, max_depth=4, random_state=42, score=0.623, total=   0.7s
[CV] class_weight=balanced, criterion=gini, max_depth=4, random_state=42 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.6s remaining:    0.0s


[CV]  class_weight=balanced, criterion=gini, max_depth=4, random_state=42, score=0.619, total=   0.7s
[CV] class_weight=balanced, criterion=gini, max_depth=4, random_state=42 


...

[CV]  class_weight=balanced, criterion=gini, max_depth=6, random_state=42, score=0.637, total=   1.1s
[CV] class_weight=balanced, criterion=gini, max_depth=6, random_state=42 


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    3.1s remaining:    0.0s


[CV]  class_weight=balanced, criterion=gini, max_depth=6, random_state=42, score=0.627, total=   1.2s
[CV] class_weight=balanced, criterion=gini, max_depth=6, random_state=42 
[CV]  class_weight=balanced, criterion=gini, max_depth=6, random_state=42, score=0.635, total=   1.1s
[CV] class_weight=balanced, criterion=gini, max_depth=8, random_state=42 
[CV]  class_weight=balanced, criterion=gini, max_depth=8, random_state=42, score=0.648, total=   1.4s
[CV] class_weight=balanced, criterion=gini, max_depth=8, random_state=42 
[CV]  class_weight=balanced, criterion=gini, max_depth=8, random_state=42, score=0.645, total=   1.5s
[CV] class_weight=balanced, criterion=gini, max_depth=8, random_state=42 
[CV]  class_weight=balanced, criterion=gini, max_depth=8, random_state=42, score=0.648, total=   1.8s
[CV] class_weight=balanced, criterion=entropy, max_depth=4, random_state=42 
[CV]  class_weight=balanced, criterion=entropy, max_depth=4, random_state=42, score=0.602, total=   1.7s
[CV] class_w

[Parallel(n_jobs=1)]: Done  18 out of  18 | elapsed:   22.8s finished


Decision Tree Best Params:
 {'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 8, 'random_state': 42}


In [ ]:
#Light GBM

lgbm


In [ ]:
#XGBoost

xgb_params = {
 "learning_rate"    : [0.05, 0.10, 0.15, 0.20, 0.25, 0.30 ] ,
 "max_depth"        : [ 3, 4, 5, 6, 8, 10, 12, 15],
 "min_child_weight" : [ 1, 3, 5, 7 ],
 "gamma"            : [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
 "colsample_bytree" : [ 0.3, 0.4, 0.5 , 0.7 ]}


xgbtuning = GridSearchCV(xgb, param_grid = xgb_params, cv = 3, verbose = 5)

xgbtuning.fit(X_train,y_train)


print(xgbbest_estimator = xgbtuning.best_estimator_)
print("XGboost Params:\n",xgbtuning.best_params_)

Fitting 3 folds for each of 3840 candidates, totalling 11520 fits
[CV] colsample_bytree=0.3, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  colsample_bytree=0.3, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, score=0.661, total=   8.9s
[CV] colsample_bytree=0.3, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    8.8s remaining:    0.0s


[CV]  colsample_bytree=0.3, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, score=0.654, total=   8.6s
[CV] colsample_bytree=0.3, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1 


...

[CV]  colsample_bytree=0.3, gamma=0.4, learning_rate=0.25, max_depth=8, min_child_weight=7, score=0.675, total=  15.6s
[CV] colsample_bytree=0.3, gamma=0.4, learning_rate=0.25, max_depth=8, min_child_weight=7 
[CV]  colsample_bytree=0.3, gamma=0.4, learning_rate=0.25, max_depth=8, min_child_weight=7, score=0.676, total=  17.6s
[CV] colsample_bytree=0.3, gamma=0.4, learning_rate=0.25, max_depth=10, min_child_weight=1 
[CV]  colsample_bytree=0.3, gamma=0.4, learning_rate=0.25, max_depth=10, min_child_weight=1, score=0.677, total=  21.5s
[CV] colsample_bytree=0.3, gamma=0.4, learning_rate=0.25, max_depth=10, min_child_weight=1 
[CV]  colsample_bytree=0.3, gamma=0.4, learning_rate=0.25, max_depth=10, min_child_weight=1, score=0.679, total=  21.6s
[CV] colsample_bytree=0.3, gamma=0.4, learning_rate=0.25, max_depth=10, min_child_weight=1 
[CV]  colsample_bytree=0.3, gamma=0.4, learning_rate=0.25, max_depth=10, min_child_weight=1, score=0.679, total=  20.2s
[CV] colsample_bytree=0.3, gamma=0.

[CV]  colsample_bytree=0.3, gamma=0.4, learning_rate=0.3, max_depth=3, min_child_weight=1, score=0.671, total=   8.7s
[CV] colsample_bytree=0.3, gamma=0.4, learning_rate=0.3, max_depth=3, min_child_weight=1 
[CV]  colsample_bytree=0.3, gamma=0.4, learning_rate=0.3, max_depth=3, min_child_weight=1, score=0.672, total=   9.1s
[CV] colsample_bytree=0.3, gamma=0.4, learning_rate=0.3, max_depth=3, min_child_weight=3 
[CV]  colsample_bytree=0.3, gamma=0.4, learning_rate=0.3, max_depth=3, min_child_weight=3, score=0.673, total=   8.8s
[CV] colsample_bytree=0.3, gamma=0.4, learning_rate=0.3, max_depth=3, min_child_weight=3 
[CV]  colsample_bytree=0.3, gamma=0.4, learning_rate=0.3, max_depth=3, min_child_weight=3, score=0.669, total=   8.7s
[CV] colsample_bytree=0.3, gamma=0.4, learning_rate=0.3, max_depth=3, min_child_weight=3 
[CV]  colsample_bytree=0.3, gamma=0.4, learning_rate=0.3, max_depth=3, min_child_weight=3, score=0.672, total=   8.5s
[CV] colsample_bytree=0.3, gamma=0.4, learning_rat

[CV]  colsample_bytree=0.3, gamma=0.4, learning_rate=0.3, max_depth=6, min_child_weight=3, score=0.677, total=  12.7s
[CV] colsample_bytree=0.3, gamma=0.4, learning_rate=0.3, max_depth=6, min_child_weight=5 
[CV]  colsample_bytree=0.3, gamma=0.4, learning_rate=0.3, max_depth=6, min_child_weight=5, score=0.675, total=  14.0s
[CV] colsample_bytree=0.3, gamma=0.4, learning_rate=0.3, max_depth=6, min_child_weight=5 
[CV]  colsample_bytree=0.3, gamma=0.4, learning_rate=0.3, max_depth=6, min_child_weight=5, score=0.675, total=  12.6s
[CV] colsample_bytree=0.3, gamma=0.4, learning_rate=0.3, max_depth=6, min_child_weight=5 
[CV]  colsample_bytree=0.3, gamma=0.4, learning_rate=0.3, max_depth=6, min_child_weight=5, score=0.677, total=  13.2s
[CV] colsample_bytree=0.3, gamma=0.4, learning_rate=0.3, max_depth=6, min_child_weight=7 
[CV]  colsample_bytree=0.3, gamma=0.4, learning_rate=0.3, max_depth=6, min_child_weight=7, score=0.676, total=  12.8s
[CV] colsample_bytree=0.3, gamma=0.4, learning_rat

[CV]  colsample_bytree=0.3, gamma=0.4, learning_rate=0.3, max_depth=12, min_child_weight=7, score=0.673, total=  22.0s
[CV] colsample_bytree=0.3, gamma=0.4, learning_rate=0.3, max_depth=12, min_child_weight=7 
[CV]  colsample_bytree=0.3, gamma=0.4, learning_rate=0.3, max_depth=12, min_child_weight=7, score=0.672, total=  22.7s
[CV] colsample_bytree=0.3, gamma=0.4, learning_rate=0.3, max_depth=12, min_child_weight=7 
[CV]  colsample_bytree=0.3, gamma=0.4, learning_rate=0.3, max_depth=12, min_child_weight=7, score=0.673, total=  26.0s
[CV] colsample_bytree=0.3, gamma=0.4, learning_rate=0.3, max_depth=15, min_child_weight=1 
[CV]  colsample_bytree=0.3, gamma=0.4, learning_rate=0.3, max_depth=15, min_child_weight=1, score=0.678, total=  32.4s
[CV] colsample_bytree=0.3, gamma=0.4, learning_rate=0.3, max_depth=15, min_child_weight=1 
[CV]  colsample_bytree=0.3, gamma=0.4, learning_rate=0.3, max_depth=15, min_child_weight=1, score=0.673, total=  30.7s
[CV] colsample_bytree=0.3, gamma=0.4, lea

[CV]  colsample_bytree=0.4, gamma=0.0, learning_rate=0.05, max_depth=5, min_child_weight=1, score=0.667, total=  13.5s
[CV] colsample_bytree=0.4, gamma=0.0, learning_rate=0.05, max_depth=5, min_child_weight=1 
[CV]  colsample_bytree=0.4, gamma=0.0, learning_rate=0.05, max_depth=5, min_child_weight=1, score=0.669, total=  13.2s
[CV] colsample_bytree=0.4, gamma=0.0, learning_rate=0.05, max_depth=5, min_child_weight=3 
[CV]  colsample_bytree=0.4, gamma=0.0, learning_rate=0.05, max_depth=5, min_child_weight=3, score=0.672, total=  12.6s
[CV] colsample_bytree=0.4, gamma=0.0, learning_rate=0.05, max_depth=5, min_child_weight=3 
[CV]  colsample_bytree=0.4, gamma=0.0, learning_rate=0.05, max_depth=5, min_child_weight=3, score=0.668, total=  12.9s
[CV] colsample_bytree=0.4, gamma=0.0, learning_rate=0.05, max_depth=5, min_child_weight=3 
[CV]  colsample_bytree=0.4, gamma=0.0, learning_rate=0.05, max_depth=5, min_child_weight=3, score=0.668, total=  12.8s
[CV] colsample_bytree=0.4, gamma=0.0, lea

In [ ]:
print("AdaBoost:\n")
print(adbcbest_estimator = grid_adbc.best_estimator_)
print("AdaBoost Best Params:\n",grid_adbc.best_params_)  

print('\n')
print('\n')

print("Bagging:\n")
print(bagbest_estimator = grid_bag.best_estimator_)
print("Bagging Best Params:\n",grid_bag.best_params_)  

print('\n')
print('\n')

print("Decision Tree:\n")
print(dtcbest_estimator = dtctuning.best_estimator_)
print("Decision Tree Best Params:\n",dtctuning.best_params_)

print('\n')
print('\n')

print("Extra Trees:\n")
print(extcbest_estimator = grid_extc.best_estimator_)
print("Extra Trees Best Params:\n",grid_extc.best_params_)  

print('\n')
print('\n')

print("Gradient Boost:\n")
print(gbcbest_estimator = gbctuning.best_estimator_)                         
print("Gradient Boost:\n",gbctuning.best_params_)

print('\n')
print('\n')

print("Random  Forest:\n")
print(rfcbest_estimator = grid_rfc.best_estimator_)
print("Random Forest:\n",grid_rfc.best_params_)

print('\n')
print('\n')

print("XGBoost:\n")
print(xgbbest_estimator = xgbtuning.best_estimator_)
print("XGboost Params:\n",xgbtuning.best_params_)


## Re-Learning W. New Parameters

In [277]:
#AdaBoost Classifier 
#AdaBoost Best Params:{'algorithm': 'SAMME.R', 'random_state': 42}

adbc2 = AdaBoostClassifier(algorithm='SAMME.R', random_state=42)
adbc2.fit(X_train, y_train)
pred_adbc2 = adbc2.predict(X_test)

#Check accuracy

print('\n')
print("AdaBoost Classifier Accuracy Score:",accuracy_score(y_test,pred_adbc2 )*100)
print("AdaBoost Classifier F1 Score:",f1_score(y_test,pred_adbc2, average="macro")*100)
print("AdaBoost Classifier Precision Score:",precision_score(y_test,pred_adbc2, average="macro")*100)
print("AdaBoost Classifier Recall Score:",recall_score(y_test, pred_adbc2, average="macro")*100) 
print("AdaBoost Classifier Cross Validation Scores:", cross_val_score(adbc2, X_train, y_train, cv=3)*100)
print("\n")
print("AdaBoost Classifier Confusion Matrix:\n", confusion_matrix(y_test,pred_adbc2))




AdaBoost Classifier Accuracy Score: 65.86353397393859
AdaBoost Classifier F1 Score: 54.48298144463484
AdaBoost Classifier Precision Score: 57.10604313495624
AdaBoost Classifier Recall Score: 65.72464537323212
AdaBoost Classifier Cross Validation Scores: [65.96471757 65.54413225 65.94135171]


AdaBoost Classifier Confusion Matrix:
 [[81271 42039]
 [ 5849 11125]]


In [279]:
#Bagging Classifier


#Bagging Best Params:{'bootstrap': 'true', 'bootstrap_features': 'true',
#'max_samples': 1000, 'n_estimators': 300, 'oob_score': 'true', 'random_state': 42}



bagc2 = BaggingClassifier(bootstrap=True, bootstrap_features=True, 
                          max_samples=1000, n_estimators=300, oob_score=True,
                          random_state=42)

bagc2.fit(X_train, y_train)
pred_bagc2 = bagc2.predict(X_test)


#Check accuracy

print('\n')
print("Bagging Classifier Accuracy Score:",accuracy_score(y_test,pred_bagc2 )*100)
print("Bagging Classifier F1 Score:",f1_score(y_test,pred_bagc2, average="macro")*100)
print("Bagging Classifier Precision Score:",precision_score(y_test,pred_bagc2, average="macro")*100)
print("Bagging Classifier Recall Score:",recall_score(y_test, pred_bagc2, average="macro")*100) 
print("Bagging Classifier Cross Validation Scores:", cross_val_score(bagc2, X_train, y_train, cv=3)*100)
print("\n")
print("Bagging Classifier Confusion Matrix:\n", confusion_matrix(y_test,pred_bagc2))



Bagging Classifier Accuracy Score: 65.66607738587436
Bagging Classifier F1 Score: 54.32501424351286
Bagging Classifier Precision Score: 57.02334846210737
Bagging Classifier Recall Score: 65.5691434347677
Bagging Classifier Cross Validation Scores: [65.93258952 65.77487003 65.91798586]


Bagging Classifier Confusion Matrix:
 [[81011 42299]
 [ 5866 11108]]


In [280]:
#Decision Tree Classifier

#Decision Tree Best Params:{'class_weight': 'balanced', 'criterion': 'gini', 
#'max_depth': 8, 'random_state'


dtc = DecisionTreeClassifier(class_weight='balanced', criterion='gini', 
                             max_depth=8, random_state=42)

dtc.fit(X_train, y_train)
pred_dtc = dtc.predict(X_test)
#Check accuracy

print('\n')
print("Decision Tree Classifier Accuracy Score:",accuracy_score(y_test,pred_dtc )*100)
print("Decision Tree Classifier F1 Score:",f1_score(y_test,pred_dtc, average="macro")*100)
print("Decision Tree Classifier Precision Score:",precision_score(y_test,pred_dtc, average="macro")*100)
print("Decision Tree Classifier Recall Score:",recall_score(y_test, pred_dtc, average="macro")*100) 
print("Decision Tree Classifier Cross Validation Scores:", cross_val_score(dtc, X_train, y_train, cv=3)*100)
print("\n")
print("Decision Tree Classifier Confusion Matrix:\n", confusion_matrix(y_test,pred_dtc))




Decision Tree Classifier Accuracy Score: 67.53870719397793
Decision Tree Classifier F1 Score: 54.96170671521916
Decision Tree Classifier Precision Score: 56.82035458360557
Decision Tree Classifier Recall Score: 64.59964514987536
Decision Tree Classifier Cross Validation Scores: [64.80810795 64.51311408 64.82855307]


Decision Tree Classifier Confusion Matrix:
 [[84439 38871]
 [ 6667 10307]]


In [281]:
#ExtraTreesClassifier

#Extra Trees Best Params:{'bootstrap': 'true', 'criterion': 'gini', 
#'max_depth': 10, 'min_samples_split': 2, 'oob_score': 'true', 'random_state': 42}


extc2 = ExtraTreesClassifier(bootstrap=True, criterion='gini', max_depth=10,
                             min_samples_split=2, oob_score=True, random_state=42)
extc2.fit(X_train, y_train)
pred_extc2 = extc2.predict(X_test)

#Check accuracy

print('\n')
print("Extra Trees Classifier Accuracy Score:",accuracy_score(y_test,pred_extc2 )*100)
print("Extra Trees Classifier F1 Score:",f1_score(y_test,pred_extc2, average="macro")*100)
print("Extra Trees Classifier Precision Score:",precision_score(y_test,pred_extc2, average="macro")*100)
print("Extra Trees Classifier Recall Score:",recall_score(y_test, pred_extc2, average="macro")*100) 
print("Extra Trees Classifier Cross Validation Scores:", cross_val_score(extc2, X_train, y_train, cv=3)*100)
print("\n")
print("Extra Trees Classifier Confusion Matrix:\n", confusion_matrix(y_test,pred_extc2))




Extra Trees Classifier Accuracy Score: 65.26261013372873
Extra Trees Classifier F1 Score: 53.58849413263456
Extra Trees Classifier Precision Score: 56.356508117108916
Extra Trees Classifier Recall Score: 64.04413880447437
Extra Trees Classifier Cross Validation Scores: [64.74093113 63.87347392 64.46930311]


Extra Trees Classifier Confusion Matrix:
 [[80955 42355]
 [ 6376 10598]]


In [282]:
#Gradient Boosting Classifier

#Gradient Boost:{'learning_rate': 0.05, 'max_depth': 8, 
#'min_samples_leaf': 1, 'n_estimators': 500, 'random_state': 42}


gbc2 = ensemble.GradientBoostingClassifier(learning_rate=0.05, max_depth=8, 
                                           min_samples_leaf=1, n_estimators=500,
                                           random_state = 42)
gbc2.fit(X_train, y_train)
pred_gbc2 = gbc2.predict(X_test)
#Check accuracy

print('\n')
print("Gradient Boosting Classifier Accuracy Score:",accuracy_score(y_test,pred_gbc2 )*100)
print("Gradient Boosting Classifier F1 Score:",f1_score(y_test, pred_gbc2, average="macro")*100)
print("Gradient Boosting Classifier Precision Score:",precision_score(y_test,pred_gbc2, average="macro")*100)
print("Gradient Boosting Classifier Recall Score:",recall_score(y_test, pred_gbc2, average="macro")*100) 
print("Gradient Boosting Classifier Cross Validation Scores:", cross_val_score(gbc2, X_train, y_train, cv=3)*100)
print("\n")
print("Gradient Boosting Classifier Confusion Matrix:\n", confusion_matrix(y_test,pred_gbc2))




Gradient Boosting Classifier Accuracy Score: 68.19309400929544
Gradient Boosting Classifier F1 Score: 56.67056655135182
Gradient Boosting Classifier Precision Score: 58.46525955868693
Gradient Boosting Classifier Recall Score: 68.41130787433087
Gradient Boosting Classifier Cross Validation Scores: [69.01688183 68.45610141 68.63134529]


Gradient Boosting Classifier Confusion Matrix:
 [[84003 39307]
 [ 5313 11661]]


In [ ]:
#Light GBM

lgbm2 = lgb.lgbmClassifier(random_state = 42)
lgbm2.fit(X_train, y_train)
pred_lgbm2 = lgbm2.predict(X_test)

print('\n')
print("LightGBM Classifier Accuracy Score:",accuracy_score(y_test,pred_lgbm2 )*100)
print("LightGBM Classifier F1 Score:",f1_score(y_test, pred_lgbm2, average="macro")*100)
print("LightGBM Classifier Precision Score:",precision_score(y_test,pred_lgbm2, average="macro")*100)
print("LightGBM Classifier Recall Score:",recall_score(y_test, pred_lgbm2, average="macro")*100) 
print("LightGBM Classifier Cross Validation Scores:", cross_val_score(lgbm2, X_train, y_train, cv=3)*100)
print("\n")
print("LightGBM Classifier Confusion Matrix:\n", confusion_matrix(y_test,pred_lgbm2))






In [253]:
#Random Forest
#Random Forest: {'criterion': 'entropy', 'max_depth': 100, 
#'max_features': 'sqrt', 'min_samples_split': 8, 'n_estimators': 300, 'random_state': 42}


rfc2 = RandomForestClassifier(criterion='entropy', max_depth=100, 
                              max_features='sqrt', min_samples_split=8, 
                              n_estimators=300, random_state=42)
rfc2.fit(X_train, y_train)
pred_rfc2 = rfc2.predict(X_test)
#Check accuracy

print("Random Forest Classifier Accuracy Score: {:0.2f}%".format(accuracy_score(y_test,
                                                                               pred_rfc2 )*100))
print("Random Forest Classifier F1 Score: {:0.2f}%".format(f1_score(y_test,
                                                                   pred_rfc2,average="macro")*100))
print("Random Forest Classifier Precision Score: {:0.2f}%".format(precision_score(y_test,
                                                                                 pred_rfc2, 
                                                                                 average="macro")*100))
print("Random Forest Classifier Recall Score: {:0.2f}%".format(recall_score(y_test, 
                                                                           pred_rfc2,
                                                                           average="macro")*100))
print("Random Forest Classifier Cross Validation Scores:", cross_val_score(rfc2, 
                                                                           X_train,
                                                                           y_train,
                                                                           cv=3)*100)
print('\n')
print("Random Forest Classifier Confusion Matrix:\n", confusion_matrix(y_test,pred_rfc2))

Random Forest Classifier Accuracy Score: 66.80%
Random Forest Classifier F1 Score: 55.77%
Random Forest Classifier Precision Score: 58.15%
Random Forest Classifier Recall Score: 68.05%
Random Forest Classifier Cross Validation Scores: [69.17460132 69.00227817 69.32647935]


Random Forest Classifier Confusion Matrix:
 [[81882 41428]
 [ 5144 11830]]


In [87]:
#finding important features
rfcfeature_importances2 = pd.DataFrame(rfc2.feature_importances_,
                                   index = X_train.columns,
                                    columns=['importance']).sort_values('importance', ascending=False)
#get features by importance
rfcfeature_importances2.importance.head(130)

lsoa_of_accident_location                      0.050156
latitude                                       0.047912
engine_capacity_cc                             0.047499
date                                           0.047262
model                                          0.046723
longitude                                      0.046659
time                                           0.044108
local_authority_district                       0.036854
1st_road_number                                0.035356
local_authority_highway                        0.032332
make                                           0.030157
x1st_point_of_impact                           0.027971
vehicle_manoeuvre                              0.027571
number_of_vehicles                             0.027261
police_force                                   0.025975
hour                                           0.025905
month                                          0.025677
driver_imd_decile                              0